In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import pathlib
from collections import Counter

import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf

E0000 00:00:1747895803.748708  117846 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747895803.753343  117846 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747895803.763642  117846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747895803.763674  117846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747895803.763676  117846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747895803.763677  117846 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32 
VOCAB_SIZE = 2300
UNITS = 256

In [4]:
path_to_file = pathlib.Path("data/vie-eng/vie.txt")
text = path_to_file.read_text(encoding = "utf-8")

lines = text.splitlines()
pairs = np.array([line.split('\t')[:2] for line in lines])
context, target = pairs[:,0], pairs[:, 1]

print(f"There're {len(lines)} tranlsaltions in the datasets")
print("The first 5 pairs")
print(pairs[:5])

There're 9428 tranlsaltions in the datasets
The first 5 pairs
[['Run!' 'Chạy!']
 ['Run!' 'Chạy đi!']
 ['Help!' 'Giúp tôi với!']
 ['Help!' 'Cứu tôi!']
 ['Jump!' 'Nhảy đi!']]


In [5]:
context_train, context_val, target_train, target_val = train_test_split(context, target, test_size = 0.2, random_state = 42)
train_raw = (
    tf.data.Dataset.from_tensor_slices((context_train, target_train))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)
val_raw = (
    tf.data.Dataset.from_tensor_slices((context_val, target_val))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

I0000 00:00:1747895807.779282  117846 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2146 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
def clean(text):
    # text = tf.strings.lower(text)
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[^a-zA-ZÀ-ỹà-ỹ?.,! ]", '')  # keep Vietnamese words
    text = tf.strings.regex_replace(text, "[?.,!]", r' \0 ')
    text = tf.strings.strip(text)
    text = tf.strings.join(["[SOS]", text, "[EOS]"], separator=" ")
    return text

In [8]:
example_vie = "Bí mật của cuộc sống là ngã bảy lần và đứng dậy tám lần – trích Nhà Giả Kim | Paulo Coelho"
ex_vie_out = clean(example_vie)
decoded_ex_vie_out = ex_vie_out.numpy().decode()
print("Example sentence")
print(example_vie,'\n')

print("Example cleaned of example sentence (Not decoded)")
print(ex_vie_out, '\n')
print("Decoded")
print(decoded_ex_vie_out)

Example sentence
Bí mật của cuộc sống là ngã bảy lần và đứng dậy tám lần – trích Nhà Giả Kim | Paulo Coelho 

Example cleaned of example sentence (Not decoded)
tf.Tensor(b'[SOS] b\xc3\xad m\xe1\xba\xadt c\xe1\xbb\xa7a cu\xe1\xbb\x99c s\xe1\xbb\x91ng l\xc3\xa0 ng\xc3\xa3 b\xe1\xba\xa3y l\xe1\xba\xa7n v\xc3\xa0 \xc4\x91\xe1\xbb\xa9ng d\xe1\xba\xady t\xc3\xa1m l\xe1\xba\xa7n  tr\xc3\xadch nh\xc3\xa0 gi\xe1\xba\xa3 kim  paulo coelho [EOS]', shape=(), dtype=string) 

Decoded
[SOS] bí mật của cuộc sống là ngã bảy lần và đứng dậy tám lần  trích nhà giả kim  paulo coelho [EOS]


In [9]:
eng_vectorizer = tf.keras.layers.TextVectorization(standardize = clean,
                                                   max_tokens = VOCAB_SIZE,
                                                   ragged = True)
eng_vectorizer.adapt(train_raw.map(lambda context, target: context))

vie_vectorizer = tf.keras.layers.TextVectorization(standardize = clean,
                                                   max_tokens = VOCAB_SIZE,
                                                   ragged = True)
vie_vectorizer.adapt(train_raw.map(lambda context, target: target))

In [10]:
print(vie_vectorizer.get_vocabulary()[:20])
vie_vectorizer.vocabulary_size()

['', '[UNK]', '[SOS]', '[EOS]', '.', 'tôi', 'không', 'tom', 'bạn', 'có', 'đã', 'là', '?', 'một', 'ấy', 'anh', 'làm', 'của', 'đó', 'sẽ']


2300

In [11]:
print(eng_vectorizer.get_vocabulary()[:20])
eng_vectorizer.vocabulary_size()

['', '[UNK]', '[SOS]', '[EOS]', '.', 'i', 'to', 'tom', 'the', 'you', '?', 'a', 'is', 'that', 'do', 'he', 'in', 'of', ',', 'this']


2300

In [12]:
eng_vectorizer(["Hello who are you"])

<tf.RaggedTensor [[2, 1182, 88, 28, 9, 3]]>

In [13]:
eng_word2id = tf.keras.layers.StringLookup(
    vocabulary = eng_vectorizer.get_vocabulary(),
    mask_token = '',
    oov_token = "[UNK]"
)

eng_id2word = tf.keras.layers.StringLookup(
    vocabulary = eng_vectorizer.get_vocabulary(),
    mask_token = '',
    oov_token = "[UNK]",
    invert = True
)

vie_id2word = tf.keras.layers.StringLookup(
    vocabulary = vie_vectorizer.get_vocabulary(),
    mask_token = '',
    oov_token = "[UNK]",
    invert = True
)

vie_id2word = tf.keras.layers.StringLookup(
    vocabulary = vie_vectorizer.get_vocabulary(),
    mask_token = '',
    oov_token = "[UNK]",
    invert = True
)

print(eng_word2id("[UNK]"))
print(eng_word2id("jump"))

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(1461, shape=(), dtype=int64)


In [14]:
def process_text(context, target):
    context = eng_vectorizer(context).to_tensor()
    target = vie_vectorizer(target)
    # Use the current word to predict the next word
    target_in = target[:, :-1].to_tensor()
    target_out = target[:, 1:].to_tensor()
    return (context, target_in), target_out

In [15]:
for _ in train_raw.take(1):
    context, output = _ 
    print(context[0])
    print(output[0])

tf.Tensor(b'Give that back to me.', shape=(), dtype=string)
tf.Tensor(b'Tr\xe1\xba\xa3 c\xc3\xa1i \xc4\x91\xc3\xb3 l\xe1\xba\xa1i cho t\xc3\xb4i.', shape=(), dtype=string)


In [16]:
train_data = train_raw.map(process_text, tf.data.AUTOTUNE)
val_data = val_raw.map(process_text, tf.data.AUTOTUNE)

In [17]:
for _ in train_data.take(1):
    a, b = _
    a1, a2 = a 
    print(a1.shape)
    print(a2.shape)
    print(b.shape)
    print(a1[0])

(32, 15)
(32, 19)
(32, 19)
tf.Tensor([  2   9  93   1  13  16 423   4   3   0   0   0   0   0   0], shape=(15,), dtype=int64)


In [18]:
def vect_to_text(vect, id2word):
    """
    Convert from vector of ids into the corresponding text

    Args:
        vect (tf.Tensor): include id of words in the sequence
        id2word (tf.keras.layers.StringLookup): invert string lookup for convert id to word

    Returns:
        str: the converted texts
    """
    words = id2word(vect)
    no_pad = tf.boolean_mask(words, words != b'')
    text = tf.strings.reduce_join(no_pad, separator = ' ')
    return text.numpy().decode()

In [19]:
for x in train_data.take(1):
    (context, target_in), target_out = x
    train_context0 = context[0]
    target_in0 = target_in[0]
    target_out0 = target_out[0]
    
    print(train_context0)
    print(vect_to_text(train_context0, eng_id2word), '\n')
    print(target_in0)
    print(vect_to_text(target_in0, vie_id2word), '\n')
    print(target_out0)
    print(vect_to_text(target_out0, vie_id2word), '\n')

tf.Tensor([  2   9 312  39  72  58  20  86   5  39  58   9   4   3   0   0], shape=(16,), dtype=int64)
[SOS] you probably know more about me than i know about you . [EOS] 

tf.Tensor(
[  2  69   9  63  27  66  55 211  64  11 211  27  55  69   4   0   0   0
   0   0   0   0], shape=(22,), dtype=int64)
[SOS] cậu có khi biết nhiều về tớ hơn là tớ biết về cậu . 

tf.Tensor(
[ 69   9  63  27  66  55 211  64  11 211  27  55  69   4   3   0   0   0
   0   0   0   0], shape=(22,), dtype=int64)
cậu có khi biết nhiều về tớ hơn là tớ biết về cậu . [EOS] 



In [20]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, units, vocab_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            input_dim = vocab_size, 
            output_dim = units, 
            mask_zero = True
        )
        self.rnn = tf.keras.layers.LSTM(
            units = units,
            return_sequences = True, 
        )
        
    def call(self, context):
        """
        Forward pass of this layer

        Args:
            context (tf.Tensor) : The sentences to translate (B, L) 
            
        Returns:
            tf.Tensor: (B, L, D): Encoded sentence to tranlsate
        """
        x = self.embedding(context)
        x = self.rnn(x)
        return x

In [21]:
encoder = Encoder(UNITS, VOCAB_SIZE)
encoder_output = encoder(context)

print(context.shape)
print(encoder_output.shape)

(32, 16)
(32, 16, 256)


I0000 00:00:1747895810.945933  117934 cuda_dnn.cc:529] Loaded cuDNN version 90300


In [22]:
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(
            key_dim = units,
            num_heads = 1
        )
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        
    def call(self, context, target):
        """Forward pass of Attention

        Args:
            context (tf.Tensor): (B, L1, D) encoded setence to translate
            target (tf.Tensor): (B, L2, D) right shifted translation 
            
        Return:
            tf.Tensor: (B, L2, D) cross attention between context and target
        """
        # query is the translation and the value is the context 
        # default key = value 
        attn_output = self.mha(
            query = target,
            value = context
        )
        x = self.add([target, attn_output])
        x = self.layernorm(x)
        return x     

In [23]:
attn = CrossAttention(UNITS)
attn_result = attn(
    encoder_output,
    tf.keras.layers.Embedding(input_dim = VOCAB_SIZE, output_dim = UNITS, mask_zero = True)(target_out)
)

print(attn_result.shape)

/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.

(32, 22, 256)


/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'cross_attention' (of type CrossAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [24]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, units, vocab_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim = units, 
            mask_zero = True
        )
        self.pre_attn_rnn = tf.keras.layers.LSTM(
            units = units, 
            return_sequences = True,
            return_state = True 
        )
        self.attention = CrossAttention(units)
        self.post_attn_rnn = tf.keras.layers.LSTM(
            units = units, 
            return_sequences = True
        )
        self.dense = tf.keras.layers.Dense(
            units = vocab_size, 
            activation = "log_softmax"
        )
    def call(self,
             context,
             target,
             state = None,
             return_state = False):
        """Forwardpass of Decodeer

        Args:
            context (tf.Tensor): (B, L1, D) encoded sentence to translate 
            target (tf.Tensor): (B, L1) the right-shited translation 
            state (list([tf.Tensor, tf.Tensor]), optional): Hidden state of the pre-attn RNN. Defaults to None.
            return_state (bool, optional): Return state of pre-attn LSTM. Defaults to False.
        """
        x = self.embedding(target)
        
        x, h, c = self.pre_attn_rnn(
            x, 
            initial_state = state
        )
        
        x = self.attention(context, x)
        x = self.post_attn_rnn(x)
        logits = self.dense(x)
        
        if return_state: return logits, [h, c]
        return logits

In [25]:
decoder = Decoder(UNITS, VOCAB_SIZE)
logits = decoder(encoder_output, target_out)

logits.shape

/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'cross_attention_1' (of type CrossAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'decoder' (of type Decoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


TensorShape([32, 22, 2300])

In [26]:
class Translator(tf.keras.Model):
    def __init__(self, units, vocab_size):
        super().__init__()
        self.encoder = Encoder(units, vocab_size)
        self.decoder = Decoder(units, vocab_size)
    def call(self, inputs):
        """Forward pass of the model

        Args:
            inputs (tuple(tf.Tensor, tf.Tensor)): (2, B, L) Tuple of context and target 
        
        Returns:
            tf.Tensor: (B, L2, V) log_softmax probabilites of predict a particular token
        """
        context, target = inputs 
        encoded_context = self.encoder(context)
        logits = self.decoder(encoded_context, target)
        return logits

In [27]:
translator = Translator(UNITS, VOCAB_SIZE)
logits = translator((context, target_in))
logits.shape

/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'cross_attention_2' (of type CrossAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/vz/miniconda3/envs/cu118/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'decoder_1' (of type Decoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


TensorShape([32, 22, 2300])

In [28]:
def masked_loss(y_true, y_pred):
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)
    
    # Check which elements of y_true are padding
    mask = tf.cast(y_true != 0, loss.dtype)
    
    loss *= mask
    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)


def masked_acc(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)
    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)
    match*= mask

    return tf.reduce_sum(match)/tf.reduce_sum(mask)


In [29]:
translator.compile(
    optimizer = "Adam",
    loss = masked_loss,
    metrics = [masked_acc, masked_loss]
)

translator.summary()

Model: "translator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_1 (Encoder)             │ ?                      │     1,114,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_1 (Decoder)             │ ?                      │     2,494,204 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,608,316 (13.76 MB)

 Trainable params: 3,608,316 (13.76 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = translator.fit(
    train_data,
    epochs = 50,    
    validation_data = val_data,
    callbacks = [tf.keras.callbacks.EarlyStopping(patience = 3),
                 tf.keras.callbacks.TensorBoard(log_dir = "./logs")]
)

Epoch 1/50
236/236 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - loss: 5.6584 - masked_acc: 0.1337 - masked_loss: 5.6584 - val_loss: 4.8399 - val_masked_acc: 0.2149 - val_masked_loss: 4.8402
Epoch 2/50
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 4.6587 - masked_acc: 0.2344 - masked_loss: 4.6587 - val_loss: 4.3844 - val_masked_acc: 0.2695 - val_masked_loss: 4.3845
Epoch 3/50
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 4.1657 - masked_acc: 0.2969 - masked_loss: 4.1657 - val_loss: 4.0606 - val_masked_acc: 0.3123 - val_masked_loss: 4.0606
Epoch 4/50
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 3.8133 - masked_acc: 0.3394 - masked_loss: 3.8133 - val_loss: 3.8033 - val_masked_acc: 0.3510 - val_masked_loss: 3.8039
Epoch 5/50
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 3.4932 - masked_acc: 0.3834 - masked_loss: 3.4932 - val_loss: 3.5951 - val_masked_acc: 0.3858 - val_masked_loss: 3.5953
Epoch 6/50
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 3.1960 - masked_acc: 0.4262 - mask

In [31]:
def generate_next_token(
    decoder,
    context, 
    next_token,
    state, 
    done, 
    temperature = 0.0
):
    # Get the logits and state from the decoder
    logits, state = decoder(context,
                            next_token,
                            state = state,
                            return_state = True)
    
    # Trim the intermediate dimension 
    logits = logits[:, -1, :]

    if temperature == 0.0: 
        next_token = tf.argmax(logits, axis = -1)
    else:
        logits = logits / temperature 
        next_token = tf.random.categorical(logits, num_samples=1)
    
    # Trim dimensions of size 1
    logits = tf.squeeze(logits)
    next_token = tf.squeeze(next_token)
    
    # Get the logit of the selected next_token
    logit = logits[next_token].numpy()
    
    # Reshape to (1,1) since this is the expected shape for text encoded as TF tensors
    next_token = tf.reshape(next_token, shape = (1, 1))
    
    # If next_token is End-of-Sentence token you are done
    if next_token == 3: # [EOS]
        done = True
    
    return next_token, logit, state, done

In [32]:
# PROCESS SENTENCE TO TRANSLATE AND ENCODE

# A sentence you wish to translate
eng_sentence = "I miss her"

# Convert it to a tensor
texts = tf.convert_to_tensor(eng_sentence)[tf.newaxis]

# Vectorize it and pass it through the encoder
context = eng_vectorizer(texts).to_tensor()
context = encoder(context)

# SET STATE OF THE DECODER
# Next token is Start-of-Sentence since you are starting fresh
next_token = tf.fill((1, 1), 2) #[SOS]

# Hidden and Cell states of the LSTM can be mocked using uniform samples
state = [tf.random.uniform((1, UNITS)), tf.random.uniform((1, UNITS))]
done = False 

next_token, logit, state, done = generate_next_token(decoder, context, next_token, state, done , temperature = 0.5)
print(f"Next token: {next_token}\nWord:{vie_id2word(next_token)}\nLogit: {logit:.4f}\nDone? {done}")


Next token: [[330]]
Word:[[b'l\xc3\xa2u']]
Logit: -15.4154
Done? False


In [33]:
def translate(model, 
              text, 
              max_length = 15,
              temperature = 0.0):
    """Translate a English sentence to Vietnamese

    Args:
        model (tf.keras.Model): The translator model
        text (str): The English context sentence
        max_length (int, optional): The maximum length of the translation. Defaults to 15.
        temperature (float, optional): Randomness of the translation. Defaults to 0.0.
    
    Returns:
        tuple(str, np.float, tf.Tensor): The translation, logit that predicted [EOS] token, and the tokenizeed translation
    """
    tokens, logits = list(), list()
    
    # convert the string to tensor
    text = tf.convert_to_tensor(text)[tf.newaxis]
    
    # vectorize the string tensor 
    context = eng_vectorizer(text).to_tensor()
    
    # get the encoded context
    context = model.encoder(context)
    

    # [SOS]
    next_token = tf.fill((1, 1), 2)
    
    # initial hidden states(h0) and cell states(c0) will be tensor of 0 with dim (1, UNITS)
    state = [tf.zeros((1, UNITS)), tf.zeros((1, UNITS))]
    
    # done flag 
    done = False 
    for i in range(max_length):
        try:
            next_token, logit, state, done = generate_next_token(
                decoder = model.decoder,
                context = context, 
                next_token = next_token,
                state = state, 
                done = done, 
                temperature = temperature
            )
        except:
            raise Exception("Problem generating the next token")

        # check done 
        if done:
            break 
            
        # append the result 
        tokens.append(next_token)
        logits.append(logit)

    # concatenate all token into a tensor 
    tokens = tf.concat(tokens, axis = - 1)
    
    # convert translated tokens to text
    translation = vect_to_text(tokens, vie_id2word)
    
    return translation, logits[-1], tokens

In [34]:
temp = 0.4
test_eng_sentence = "Please let me go"

translation, logit, tokens = translate(translator, test_eng_sentence, temperature = temp)


print(f"Temperature: {temp}")
print(f"Original sentence: {test_eng_sentence}")
print(f"Translation: {translation}")
print(f"Translation tokens:{tokens}")
print(f"Logit: {logit}")

Temperature: 0.4
Original sentence: Please let me go
Translation: làm tôi trả tiền tôi đi .
Translation tokens:[[ 16   5 239 103   5  22   4]]
Logit: -5.307931900024414


In [35]:
def generate_samples(model, text, n_samples = 4, temperature = 0.6):
    """
    Generate n tranlsation for text

    Args:
        model (tf.keras.Model): The translate model
        text (str): The English context sentence
        n_samples (int, optional): Numer of samples need to be generated. Defaults to 4.
        temperature (float, optional): Randomess. Defaults to 0.6.

    Return: 
        tuple(list, float): list of setences (tokenized) and its log probs
    """
    samples, log_probs = list(), list()
    for _  in range(n_samples):
        translation, log_p, sample = translate(model, text, temperature = temperature)
        samples.append(np.squeeze(sample.numpy().tolist()))
        log_probs.append(log_p)
    return samples, log_probs

In [36]:
samples, log_probs = generate_samples(translator, test_eng_sentence)

for s, l in zip(samples, log_probs):
    print(f"Translated tensor: {s} has logit: {l:.3f}")

Translated tensor: [ 16  29   5  29   5  54 334 284 209   4] has logit: -3.174
Translated tensor: [ 85 116   5 273   4] has logit: -1.283
Translated tensor: [ 85  29   5  22 129   4] has logit: -2.125
Translated tensor: [ 85  29   5 222  22   4] has logit: -0.908


In [37]:
def rouge1_similarity(candidate, reference):
    """
    Compmute ROUGE1 score between 2 token lists

    Args:
        candidate (list[int]): Tokenized candidate translation
        reference (list[int]): Tokenized reference translation
    
    Return:
        float: Overlap between the two token lists
    """
    candiate_word_counts = Counter(candidate)
    reference_word_counts = Counter(reference) 
    overlap = 0
    
    for token in candidate: 
        can_token_count = candiate_word_counts[token]
        ref_token_count = reference_word_counts[token]
        overlap += min(ref_token_count, can_token_count)
    
    precision = overlap / sum(candiate_word_counts.values())
    recall = overlap / sum(reference_word_counts.values())
    
    f1_score = 0 # if precision + recall = 0 
    if precision + recall != 0 : 
        f1_score = 2 * precision * recall / (precision + recall)
    
    return f1_score

In [38]:
l1 = [1, 2, 3]
l2 = [1, 2, 3, 4]
print(rouge1_similarity(l1, l2))

0.8571428571428571


In [39]:
def average_overlap(samples):
    """
    Compute the mean of each candiate sentence in the samples

    Args:
        samples (list[list[int]]): Tokenized version of translated sentences
    
    Returns:
    list[float]: Scores at each index
    """
    n_samples = len(samples)
    scores = []
    for i1, c1 in enumerate(samples):
        s = 0 # save the current mean score
        for i2, c2 in enumerate(samples):
            if i2 == i1: continue 
            s += rouge1_similarity(c1, c2)
        
        # get average 
        s /= (n_samples - 1)
        
        # round to 3 decimal points 
        s = round(s, 3)
        
        # save to dict 
        scores.append(s)
    
    return scores
        

In [40]:
l1 = [1, 2, 3]
l2 = [1, 2, 4]
l3 = [1, 2, 4, 5]

print(average_overlap([l1, l2, l3]))

[0.619, 0.762, 0.714]


In [41]:
def weighted_avg_overlap(samples,
                         log_probs):
    """
    Compute the weighted average of each candidate sentence in samples

    Args:
        samples (list[list[int]]): Tokenized version of translated sentences
        log_probs (list[int]): log prob of each sentence
        
    Returns:
    list[float]: Scores at each index
    """
    n_samples = len(samples)
    scores = []
    for i1, c1 in enumerate(samples):
        s = 0 # save the current mean score
        
        for i2, (c2, c2_logp) in enumerate(zip(samples, log_probs)):
            if i2 == i1: continue 
            
            # convert log prob to linear scale 
            c2_p = float(np.exp(c2_logp))
            
            # compute score
            s += rouge1_similarity(c1, c2)
            
            # weight for it 
            s *= c2_p
        
        # get average 
        s /= (n_samples - 1)
        
        # round to 3 decimal points 
        s = round(s, 3)
        
        # save to dict 
        scores.append(s)
    
    return scores

In [42]:
l1 = [1, 2, 3]
l2 = [1, 2, 4]
l3 = [1, 2, 4, 5]
log_probs = [0.4, 0.2, 0.5]

print(weighted_avg_overlap([l1, l2, l3], log_probs))

[1.142, 1.526, 1.044]


In [43]:
def mbr_decode(model,
               text,
               n_samples = 5, 
               temperature = 0.6):
    # generatre samples
    samples, log_probs = generate_samples(model, text, n_samples, temperature)
    
    # compute scores
    scores = weighted_avg_overlap(samples, log_probs)
    
    # get the highest score translation 
    id = np.argmax(scores)
    translation = samples[id]
    
    return translation, samples

In [44]:
translation, candidates = mbr_decode(translator, test_eng_sentence, 5)
print("Context sentence")
print(test_eng_sentence, '\n')

print("Tranlsation candidates:")
for c in candidates:
    print(vect_to_text(c, vie_id2word))

print("\nSelected translation")
print(vect_to_text(translation, vie_id2word))

Context sentence
Please let me go 

Tranlsation candidates:
hãy cho tôi thêm năm .
hãy cho tôi suy thêm tuần hôm nay .
làm cho tôi bình nửa thế trời tò diện .
làm cho tôi chút đi .
làm tôi hãy trả việc tôi đi lạc đa .

Selected translation
làm cho tôi chút đi .
